<a href="https://colab.research.google.com/github/yellowstarhx/EE599-Project/blob/master/vgg_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# Reference: https://blog.csdn.net/rocachilles/article/details/87894808
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from skimage import io, transform
import glob
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)

test_path = '/content/drive/My Drive/test_vgg/'
model_path = "/content/drive/My Drive/model/vgg.ckpt"

w = 224
h = 224
c = 3
n_class = 40

def read_img(path):
    cate   = [path + x for x in os.listdir(path) if os.path.isdir(path + x)]
    imgs   = []
    labels = []
    label_list = np.eye(n_class)
    for idx, folder in enumerate(cate):            #search folder
        for im in glob.glob(folder + '/*.jpg'):    #change doc type if necessary
            img = io.imread(im)
            img = transform.resize(img, (w, h, c))
            imgs.append(img)                        # (sum,224,224,3)
            labels.append(label_list[idx])          # (sum,4)         
        for im in glob.glob(folder + '/*.png'):    #change doc type if necessary
            img = io.imread(im)
            img = transform.resize(img, (w, h, c))
            imgs.append(img)                        # (sum,224,224,3)
            labels.append(label_list[idx])          # (sum,4)                         
    return np.asarray(imgs, np.float32), np.asarray(labels, np.float32)

data_t, label_t = read_img(test_path)
   
s_test = data_t.shape[0]                        
arr = np.arange(s_test)                    
# np.random.shuffle(arr)                           # random sequence
x_val = data_t[arr]
y_val = label_t[arr]

#------------------vgg16 structure----------------
 
x = tf.placeholder(tf.float32, shape=[None, h, w, c])
y = tf.placeholder(tf.float32, shape=[None, n_class])     

#----------------- conv1 ------------------------

w_conv1_1 = tf.Variable(tf.truncated_normal([3, 3, 3, 64], stddev=0.1))
b_conv1_1 = tf.Variable(tf.constant(0.1, shape=[64]))
L_conv1_1 = tf.nn.relu(tf.nn.conv2d(x, w_conv1_1,strides=[1, 1, 1, 1], padding='SAME') + b_conv1_1)

w_conv1_2 = tf.Variable(tf.truncated_normal([3, 3, 64, 64], stddev=0.1))
b_conv1_2 = tf.Variable(tf.constant(0.1, shape=[64]))
L_conv1_2 = tf.nn.relu(tf.nn.conv2d(L_conv1_1, w_conv1_2,strides=[1, 1, 1, 1], padding='SAME') + b_conv1_2)

L_pool1 = tf.nn.max_pool(L_conv1_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#----------------- conv2 ------------------------

w_conv2_1 = tf.Variable(tf.truncated_normal([3, 3, 64, 128], stddev=0.1))
b_conv2_1 = tf.Variable(tf.constant(0.1, shape=[128]))
L_conv2_1 = tf.nn.relu(tf.nn.conv2d(L_pool1, w_conv2_1,strides=[1, 1, 1, 1], padding='SAME') + b_conv2_1)

w_conv2_2 = tf.Variable(tf.truncated_normal([3, 3, 128, 128], stddev=0.1))
b_conv2_2 = tf.Variable(tf.constant(0.1, shape=[128]))
L_conv2_2 = tf.nn.relu(tf.nn.conv2d(L_conv2_1, w_conv2_2,strides=[1, 1, 1, 1], padding='SAME') + b_conv2_2)

L_pool2 = tf.nn.max_pool(L_conv2_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#-------------------conv3 -------------------------

w_conv3_1 = tf.Variable(tf.truncated_normal([3, 3, 128, 256], stddev=0.1))
b_conv3_1 = tf.Variable(tf.constant(0.1, shape=[256]))
L_conv3_1 = tf.nn.relu(tf.nn.conv2d(L_pool2, w_conv3_1,strides=[1, 1, 1, 1], padding='SAME') + b_conv3_1)


w_conv3_2 = tf.Variable(tf.truncated_normal([3, 3, 256, 256], stddev=0.1))
b_conv3_2 = tf.Variable(tf.constant(0.1, shape=[256]))
L_conv3_2 = tf.nn.relu(tf.nn.conv2d(L_conv3_1, w_conv3_2,strides=[1, 1, 1, 1], padding='SAME') + b_conv3_2)


w_conv3_3 = tf.Variable(tf.truncated_normal([3, 3, 256, 256], stddev=0.1))
b_conv3_3 = tf.Variable(tf.constant(0.1, shape=[256]))
L_conv3_3 = tf.nn.relu(tf.nn.conv2d(L_conv3_2, w_conv3_3,strides=[1, 1, 1, 1], padding='SAME') + b_conv3_3)

L_pool3 = tf.nn.max_pool(L_conv3_3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#---------------------- conv4 -----------------------------------

w_conv4_1 = tf.Variable(tf.truncated_normal([3, 3, 256, 512], stddev=0.1))
b_conv4_1 = tf.Variable(tf.constant(0.1, shape=[512]))
L_conv4_1 = tf.nn.relu(tf.nn.conv2d(L_pool3, w_conv4_1,strides=[1, 1, 1, 1], padding='SAME') + b_conv4_1)


w_conv4_2 = tf.Variable(tf.truncated_normal([3, 3, 512, 512], stddev=0.1))
b_conv4_2 = tf.Variable(tf.constant(0.1, shape=[512]))
L_conv4_2 = tf.nn.relu(tf.nn.conv2d(L_conv4_1, w_conv4_2,strides=[1, 1, 1, 1], padding='SAME') + b_conv4_2)


w_conv4_3 = tf.Variable(tf.truncated_normal([3, 3, 512, 512], stddev=0.1))
b_conv4_3 = tf.Variable(tf.constant(0.1, shape=[512]))
L_conv4_3 = tf.nn.relu(tf.nn.conv2d(L_conv4_2, w_conv4_3,strides=[1, 1, 1, 1], padding='SAME') + b_conv4_3)

L_pool4 = tf.nn.max_pool(L_conv4_3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#---------------------- conv5 -----------------------------------

w_conv5_1 = tf.Variable(tf.truncated_normal([3, 3, 512, 512], stddev=0.1))
b_conv5_1 = tf.Variable(tf.constant(0.1, shape=[512]))
L_conv5_1 = tf.nn.relu(tf.nn.conv2d(L_pool4, w_conv5_1,strides=[1, 1, 1, 1], padding='SAME') + b_conv5_1)


w_conv5_2 = tf.Variable(tf.truncated_normal([3, 3, 512, 512], stddev=0.1))
b_conv5_2 = tf.Variable(tf.constant(0.1, shape=[512]))
L_conv5_2 = tf.nn.relu(tf.nn.conv2d(L_conv5_1, w_conv5_2,strides=[1, 1, 1, 1], padding='SAME') + b_conv5_2)


w_conv5_3 = tf.Variable(tf.truncated_normal([3, 3, 512, 512], stddev=0.1))
b_conv5_3 = tf.Variable(tf.constant(0.1, shape=[512]))
L_conv5_3 = tf.nn.relu(tf.nn.conv2d(L_conv5_2, w_conv5_3,strides=[1, 1, 1, 1], padding='SAME') + b_conv5_3)

L_pool5 = tf.nn.max_pool(L_conv5_3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#------------------------ fully connected 6 -------------------------------------
shape6 = int(np.prod(L_pool5.get_shape()[1:]))   
w_fc6 = tf.Variable(tf.truncated_normal([shape6, 4096], stddev=0.1))
b_fc6 = tf.Variable(tf.constant(0.1, shape=[4096]))
f_fc6 = tf.reshape(L_pool5, [-1, shape6])
L_fc6 = tf.nn.relu(tf.matmul(f_fc6, w_fc6) + b_fc6)

#------------------------  fully connected 7  ----------------------------

# ---------------------- Drop --------------------------
keep_prob = tf.placeholder(tf.float32)
d_fc7 = tf.nn.dropout(L_fc6, keep_prob)

w_fc7 = tf.Variable(tf.truncated_normal([4096, 4096], stddev=0.1))
b_fc7 = tf.Variable(tf.constant(0.1, shape=[4096]))
L_fc7 = tf.nn.relu(tf.matmul(d_fc7, w_fc7) + b_fc7)

#------------------------ fully connected 8 ------------------------------------

w_fc8 = tf.Variable(tf.truncated_normal([4096, n_class], stddev=0.1))
b_fc8 = tf.Variable(tf.constant(0.1, shape=[n_class]))
L_fc8 = tf.matmul(L_fc7, w_fc8) + b_fc8

#-------------------- final output -------------------------------

# y_conv = tf.nn.softmax(L_fc8)
y_conv = L_fc8
y_ = tf.nn.softmax(L_fc8)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))
train_step = tf.train.AdamOptimizer((1e-4)).minimize(cross_entropy)
prediction_cls = tf.argmax(y_conv,1) #the actual class
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# --------------- save the model ----------------------------
saver = tf.train.Saver()

#------------------ run --------------------------------
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
#     sess.run(tf.global_variables_initializer())
    print ("Input %s images, %s labels" % (s_test, s_test))
    saver.restore(sess,model_path)
    val_acc, val_loss, val_pre = sess.run([accuracy, cross_entropy, prediction_cls],feed_dict={x: x_val, y: y_val, keep_prob: 1.0})
    print('Testing accuracy %s' % (val_acc))
    print("Predicted class:")
    print(val_pre)
print("actual class:")
print(arr)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Input 40 images, 40 labels
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/model/vgg.ckpt
Testing accuracy 0.525
Predicted class:
[ 0 20 33  3 22 33 33 33  8 23 15 11 12 18 14 15 15 15 18 19 20 21 22 18
 24 22 15 27 28 39 30 33 37 33 34 39 36 39 38 39]
actual class:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
